In [ ]:
!pip install qiskit
!pip install qiskit-aqua
!pip install qiskit-finance
#!pip install iqx
%matplotlib inline


###Improving Variational Quantum Optimization using CVaR
##Introduction

This notebook shows how to use the Conditional Value at Risk (CVaR) objective function introduced in [1] within the variational quantum optimization algorithms provided by Qiskit. Particularly, it is shown how to setup the MinimumEigenOptimizer using QAOA accordingly. For a given set of shots with corresponding objective values of the considered optimization problem, the CVaR with confidence level α∈[0,1] is defined as the average of the α best shots. Thus, α=1 corresponds to the standard expected value, while α=0 corresponds to the minimum of the given shots, and α∈(0,1) is a tradeoff between focusing on better shots, but still applying some averaging to smoothen the optimization landscape.

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider,YahooDataProvider
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua.operators.expectations import CVaRExpectation,PauliExpectation
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit import execute, Aer
from qiskit.utils import algorithm_globals
from docplex.mp.model import Model

In [ ]:
algorithm_globals.random_seed = 123456

### Portfolio Optimization
In the following we define a problem instance for portfolio optimization 

In [ ]:
# prepare problem instance
n = 5            # number of assets (= number of qubits)
q = 0.5          # risk factor
budget = n // 2  # budget
penalty = 2*n    # scaling of penalty term

In [ ]:
# Generate expected return and covariance matrix from (random) time-series
stocks=["XOM","BAC","IBM","PFE","TSLA"]

# data = RandomDataProvider(tickers=stocks,
#                  start=datetime.datetime(2016,1,1),
#                  end=datetime.datetime(2016,1,30))
data = YahooDataProvider(tickers=stocks,
                 start=datetime.datetime(2015,1,1),
                 end=datetime.datetime(2020,12,31))
data.run()
mu = data.get_period_return_mean_vector()
sigma = data.get_period_return_covariance_matrix()

In [ ]:
print("MU expected return", mu)
print("SIGMA is covariance", sigma)
print("Stock tickers", stocks)

MU expected return [-0.00020049  0.00063765  0.00011569  0.00038236  0.00243455]
SIGMA is covariance [[3.05034928e-04 2.35433080e-04 1.58216451e-04 9.93862618e-05
  1.48357211e-04]
 [2.35433080e-04 4.46155354e-04 1.96199726e-04 1.33807957e-04
  2.01145479e-04]
 [1.58216451e-04 1.96199726e-04 2.50413471e-04 1.04917942e-04
  1.48370441e-04]
 [9.93862618e-05 1.33807957e-04 1.04917942e-04 1.92167591e-04
  8.62473939e-05]
 [1.48357211e-04 2.01145479e-04 1.48370441e-04 8.62473939e-05
  1.20799074e-03]]
Stock tickers ['XOM', 'BAC', 'IBM', 'PFE', 'TSLA']


In [ ]:
# create docplex model
mdl = Model('portfolio_optimization')
x = mdl.binary_var_list('x{}'.format(i) for i in range(n))
objective = mdl.sum([mu[i]*x[i] for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*x[i]*x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)
mdl.add_constraint(mdl.sum(x[i] for i in range(n)) == budget)

# case to
qp = QuadraticProgram()
qp.from_docplex(mdl)

In [ ]:
# solve classically as reference
opt_result = MinimumEigenOptimizer(NumPyMinimumEigensolver(qp))
opt_result

In [ ]:
# we convert the problem to an unconstrained problem for further analysis,
# otherwise this would not be necessary as the MinimumEigenSolver would do this
# translation automatically
linear2penalty = LinearEqualityToPenalty(penalty=penalty)
qp = linear2penalty.convert(qp)
_, offset = qp.to_ising()

## Minimum Eigen Optimizer using QAOA

In [ ]:
# set classical optimizer
maxiter = 100
optimizer = COBYLA(maxiter=maxiter)


# set variational ansatz
var_form = RealAmplitudes(n, reps=1)
m = var_form.num_parameters

# set backend
backend_name = 'qasm_simulator'  # use this for QASM simulator
# backend_name = 'statevector_simulator'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25]  # confidence levels to be evaluated

In [ ]:
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [-(obj + offset)]

# loop over all given alpha values
for alpha in alphas:

    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
    cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373

    # initialize QAOA using CVaR
    qaoa = QAOA(expectation=cvar_exp, optimizer=optimizer, quantum_instance=backend,callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha))

    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(qaoa)

    # solve problem
    results[alpha] = opt_alg.solve(qp)

    # print results
    print('alpha = {}:'.format(alpha))
    print(results[alpha])
    print()

AttributeError: ignored

In [ ]:
# plot resulting history of objective values
plt.figure(figsize=(10, 5))
plt.plot([0, maxiter], [opt_result.fval, opt_result.fval], 'r--', linewidth=2, label='optimum')
for alpha in alphas:
    plt.plot(objectives[alpha], label='alpha = %.2f' % alpha, linewidth=2)
plt.legend(loc='lower right', fontsize=14)
plt.xlim(0, maxiter)
plt.xticks(fontsize=14)
plt.xlabel('iterations', fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('objective value', fontsize=14)
plt.show()

In [ ]:
# evaluate and sort all objective values
objective_values = np.zeros(2**n)
for i in range(2**n):
    x_bin = ('{0:0%sb}' % n).format(i)
    x = [0 if x_ == '0' else 1 for x_ in reversed(x_bin)]
    objective_values[i] = qp.objective.evaluate(x)
ind = np.argsort(objective_values)

# evaluate final optimal probability for each alpha
probabilities = np.zeros(len(objective_values))
for alpha in alphas:
    if backend_name == 'qasm_simulator':
        counts = results[alpha].min_eigen_solver_result.eigenstate
        shots = sum(counts.values())
        for key, val in counts.items():
            i = int(key, 2)
            probabilities[i] = val / shots
    else:
        probabilities = np.abs(results[alpha].min_eigen_solver_result.eigenstate)**2
    print('optimal probabilitiy (alpha = %.2f):  %.4f' % (alpha, probabilities[ind][-1:]))

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

##Benchmarking QAOA and VQE solutions to the mean variance portfolio problem with real data

In [ ]:
!pip install qiskit
!pip install qiskit-finance
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.finance import QiskitFinanceError
from qiskit.finance.data_providers import *
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime
import json
import http.client


In [ ]:
def get_vars(stocks, risk_factor):
    data = YahooDataProvider(tickers = stocks, start=datetime.datetime(2015, 1, 1), end=datetime.datetime(2020, 12, 31))
    data.run()
    # set number of equities to the number of stocks, num_assets = qubits
    num_assets = len(stocks)
    mu = data.get_period_return_mean_vector()
    sigma = data.get_period_return_covariance_matrix()
    
    # Covariant matrix plot -- shows correlation between equities
    print("Covariance plot")
    plt.imshow(sigma)
    plt.show()
    
    # set the risk factor
    q = risk_factor
    # set budget
    budget = num_assets // 2 
    # scaling of budget penalty term will be dependant on the number of assets
    penalty = num_assets 
    
    # Retrieve Hamiltonian
    qubitOp, offset = portfolio.get_operator(mu, sigma, q, budget, penalty)
    
    return num_assets, mu, sigma, q, budget, penalty, qubitOp, offset

##Data processing functions

In [ ]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def selection_to_picks(num_assets, selection):
    purchase = []
    for i in range(num_assets):
        if selection[i] == 1:
            purchase.append(stocks[i])
    return purchase

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, mu, sigma, q, budget, penalty)
    print(f"!!! Optimal: portfolio holdings !!!")
    return selection

##QAOA implementation

In [ ]:
def qaoa(qubitOp):
    backend = Aer.get_backend('statevector_simulator')
    seed = 50
    cobyla = COBYLA()
    cobyla.set_options(maxiter=250)
    qaoa = QAOA(qubitOp, cobyla, 3)

    qaoa.random_seed = seed
    quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
    results= qaoa.run(quantum_instance)
    selection = print_result(results)
    print(selection_to_picks(num_assets, selection))

##Classical Eigensolver

In [ ]:
def numpyEigensolver(qubitOp):
    selections = []
    exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
    result = exact_eigensolver.run()

    selection_1 = print_result(result)
    print(selection_to_picks(num_assets, selection_1))

##VQE

In [ ]:
def vqe(qubitOp):
    backend = Aer.get_backend('statevector_simulator')
    seed = 50

    cobyla = COBYLA()
    cobyla.set_options(maxiter=500)
    ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
    vqe = VQE(qubitOp, ry, cobyla)
    vqe.random_seed = seed

    quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

    result = vqe.run(quantum_instance)

    selection_2 = print_result(result)
    print(selection_to_picks(num_assets, selection_2))


##Benchmarking

First Trial

In [ ]:
###INSERT STOCK TICKERS IN STOCKS ARRAY ###
stocks = ["MSFT", "BA", "CVS", "IBM", "CF", "BIIB", "XOM", "V", "ADBE", "PEP"]
###---###
num_assets, mu, sigma, q, budget, penalty, qubitOp, offset = get_vars(stocks, 0.7)

In [ ]:
#Classical Benchmark
numpyEigensolver(qubitOp)

In [ ]:
# VQE
vqe(qubitOp)

In [ ]:

# QAOA
qaoa(qubitOp)

##Another Trial

In [ ]:
###INSERT STOCK TICKERS IN STOCKS ARRAY ###
stocks = ['TSLA', 'MSFT', 'GOOG', 'PFE', 'AAPL', 'AMZN', 'SPY', 'ICLN', 'XOM', 'WBA']
###---###
num_assets, mu, sigma, q, budget, penalty, qubitOp, offset = get_vars(stocks, 0.7)

In [ ]:
#Classical Benchmark
numpyEigensolver(qubitOp)

In [ ]:
# VQE
vqe(qubitOp)

In [ ]:
# QAOA
qaoa(qubitOp)

##One more trial

In [ ]:

###INSERT STOCK TICKERS IN STOCKS ARRAY ###
stocks = ['ATVI', 'MA', 'MCD', 'PLD', 'ETR', 'HBI', 'NEE', 'JPM', 'VZ', 'T']
###---###
num_assets, mu, sigma, q, budget, penalty, qubitOp, offset = get_vars(stocks, 0.7)

In [ ]:
#Classical Benchmark
numpyEigensolver(qubitOp)

In [ ]:
# VQE
vqe(qubitOp)

In [ ]:

# QAOA
qaoa(qubitOp)